In [2]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import matplotlib.pyplot as plt
import random

import cornac
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality
from cornac.data.text import BaseTokenizer
from cornac.models import CTR
from cornac.data import Reader
from cornac.metrics import MAE,RMSE,MSE,FMeasure,Precision,Recall,NDCG,NCRR,MRR,AUC,MAP

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


In [3]:
amazon_clothing_category = pd.read_json('/Users/pigr/Desktop/cornac/cornac/dataset/amazon_clothing_category.json', typ='series')
amazon_clothing_category = amazon_clothing_category.to_frame().reset_index()
amazon_clothing_category.columns = ['itemId', 'category']
amazon_clothing_category.head()

FileNotFoundError: File /Users/pigr/Desktop/cornac/cornac/dataset/amazon_clothing_category.json does not exist

In [68]:
print(amazon_clothing_category.loc[0])

itemId                                             B00001TOXD
category    [Clothing, Shoes & Jewelry, Costumes & Accesso...
Name: 0, dtype: object


In [4]:
# amazon_clothing dataset
docs_clothing, item_ids_clothing = cornac.datasets.amazon_clothing.load_text()
feedback_clothing = cornac.datasets.amazon_clothing.load_feedback(reader=Reader(item_set=item_ids_clothing))
clothing_item_text_modality = TextModality(
    corpus=docs_clothing,
    ids=item_ids_clothing,
    tokenizer=BaseTokenizer(sep=" ", stop_words="english"),
    max_vocab=8000,
    max_doc_freq=0.5,
)
# Define an evaluation method to split feedback into train and test sets
clothing_ratio_split = RatioSplit(
    data=feedback_clothing,
    test_size=0.2,
    exclude_unknowns=True,
    item_text=clothing_item_text_modality,
    verbose=True,
    seed=123,
    rating_threshold=1,
)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 5081
Number of items = 3326
Number of ratings = 10951
Max rating = 5.0
Min rating = 1.0
Global mean = 4.3
---
Test data:
Number of users = 1868
Number of items = 1453
Number of ratings = 2200
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 5081
Total items = 3326


### k=10

In [6]:
# Instantiate HFT model
hft_k10 = cornac.models.HFT(
    k=10,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_k10], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.7492 | 1.2143 | 1.1020 | 0.5053 | 0.0003 | 0.0020 | 0.0024 |  0.0009 |  0.0021 |       0.0002 |    0.0075 |   18.5778 |   3.3744



In [16]:
data1 = {"model":"HFT",
        "data":"amazon clothing",
        "k":10,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.7492,"MSE":1.2143,"RMSE":1.1020,"AUC":0.5053,"F1":0.0003,"MAP":0.0020,"MRR":0.0024,"NCRR":0.0009,"NDCG":0.0021,"Precision":0.0002,"Recall":0.0075,
        "category diversity":0}

### k=50

In [26]:
# Instantiate HFT model
hft_k50 = cornac.models.HFT(
    k=50,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_k50], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.7426 | 1.1828 | 1.0876 | 0.5061 | 0.0004 | 0.0019 | 0.0024 |  0.0009 |  0.0022 |       0.0002 |    0.0083 |   27.6335 |   2.6908



In [17]:
data2 = {"model":"HFT",
        "data":"amazon clothing",
        "k":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.7426,"MSE":1.1828,"RMSE":1.0876,"AUC":0.5061,"F1":0.0004,"MAP":0.0019,"MRR":0.0024,"NCRR":0.0009,"NDCG":0.0022,"Precision":0.0002,"Recall":0.0083,
        "category diversity":0}

### k=100

In [43]:
# Instantiate HFT model
hft_k100 = cornac.models.HFT(
    k=100,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_k100], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.7508 | 1.2095 | 1.0998 | 0.5035 | 0.0003 | 0.0020 | 0.0024 |  0.0009 |  0.0019 |       0.0001 |    0.0062 |   54.4312 |   3.0548



In [18]:
data3 = {"model":"HFT",
        "data":"amazon clothing",
        "k":100,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.7508,"MSE":1.2095,"RMSE":1.0998,"AUC":0.5035,"F1":0.0003,"MAP":0.0020,"MRR":0.0024,"NCRR":0.0009,"NDCG":0.0019,"Precision":0.0001,"Recall":0.0062,
        "category diversity":0}

### k=200

In [70]:
# Instantiate HFT model
hft_k200 = cornac.models.HFT(
    k=200,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_k200], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.7359 | 1.1497 | 1.0723 | 0.5071 | 0.0003 | 0.0019 | 0.0024 |  0.0008 |  0.0020 |       0.0002 |    0.0072 |   75.9026 |   3.1388



In [19]:
data4 = {"model":"HFT",
        "data":"amazon clothing",
        "k":200,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.7359,"MSE":1.1497,"RMSE":1.0723,"AUC":0.5071,"F1":0.0003,"MAP":0.0019,"MRR":0.0024,"NCRR":0.0008,"NDCG":0.0020,"Precision":0.0002,"Recall":0.0072,
        "category diversity":0}

### k=300

In [72]:
# Instantiate HFT model
hft_k300 = cornac.models.HFT(
    k=300,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_k300], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.7391 | 1.1584 | 1.0763 | 0.5036 | 0.0003 | 0.0019 | 0.0024 |  0.0009 |  0.0020 |       0.0002 |    0.0070 |   96.4651 |   3.4485



In [20]:
data5 = {"model":"HFT",
        "data":"amazon clothing",
        "k":300,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.7391,"MSE":1.1584,"RMSE":1.0763,"AUC":0.5036,"F1":0.0003,"MAP":0.0019,"MRR":0.0024,"NCRR":0.0009,"NDCG":0.0020,"Precision":0.0002,"Recall":0.0070,
        "category diversity":0}

### k=50,lambda_text=0.1

In [5]:
# Instantiate HFT model
hft_lamText01_k50 = cornac.models.HFT(
    k=50,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_lamText01_k50], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.7482 | 1.1211 | 1.0588 | 0.5110 | 0.0003 | 0.0021 | 0.0025 |  0.0009 |  0.0018 |       0.0001 |    0.0059 |   39.0255 |   3.3174



In [6]:
data6 = {"model":"HFT",
        "data":"amazon clothing",
        "k":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.1,
        "l2_reg":0.001,
        "MAE":0.7482,"MSE":1.1211,"RMSE":1.0588,"AUC":0.5110,"F1":0.0003,"MAP":0.0021,"MRR":0.0025,"NCRR":0.0009,"NDCG":0.0018,"Precision":0.0001,"Recall":0.0059,
        "category diversity":0}

### k=50, lambda_text=1

In [7]:
# Instantiate HFT model
hft_lamText1_k50 = cornac.models.HFT(
    k=50,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_lamText1_k50], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.8140 | 1.3782 | 1.1739 | 0.5359 | 0.0022 | 0.0069 | 0.0078 |  0.0054 |  0.0135 |       0.0011 |    0.0469 |   42.8968 |   3.0836



In [8]:
data7 = {"model":"HFT",
        "data":"amazon clothing",
        "k":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":1,
        "l2_reg":0.001,
        "MAE":0.8140,"MSE":1.3782,"RMSE":1.1739,"AUC":0.5359,"F1":0.0022,"MAP":0.0069,"MRR":0.0078,"NCRR":0.0054,"NDCG":0.0135,"Precision":0.0011,"Recall":0.0469,
        "category diversity":0}

### k=50,lambda_text=0.1,l2_reg=0.01

In [9]:
# Instantiate HFT model
hft_l2001_lamText01_k50 = cornac.models.HFT(
    k=50,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.01,
    lambda_text=0.1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_l2001_lamText01_k50], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.7520 | 1.1127 | 1.0549 | 0.5113 | 0.0003 | 0.0021 | 0.0024 |  0.0009 |  0.0018 |       0.0001 |    0.0064 |   35.4338 |   2.8085



In [10]:
data8 = {"model":"HFT",
        "data":"amazon clothing",
        "k":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.1,
        "l2_reg":0.01,
        "MAE":0.7520,"MSE":1.1127,"RMSE":1.0549,"AUC":0.5113,"F1":0.0003,"MAP":0.0021,"MRR":0.0024,"NCRR":0.0009,"NDCG":0.0018,"Precision":0.0001,"Recall":0.0064,
        "category diversity":0}

### k=50,lambda_text=0.1,l2_reg=0.1

In [11]:
# Instantiate HFT model
hft_l201_lamText01_k50 = cornac.models.HFT(
    k=50,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.1,
    lambda_text=0.1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_l201_lamText01_k50], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.7397 | 1.1815 | 1.0870 | 0.5096 | 0.0004 | 0.0020 | 0.0024 |  0.0009 |  0.0021 |       0.0002 |    0.0080 |   33.5637 |   2.8060



In [12]:
data9 = {"model":"HFT",
        "data":"amazon clothing",
        "k":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.1,
        "l2_reg":0.1,
        "MAE":0.7397,"MSE":1.1815,"RMSE":1.0870,"AUC":0.5096,"F1":0.0004,"MAP":0.0020,"MRR":0.0024,"NCRR":0.0009,"NDCG":0.0021,"Precision":0.0002,"Recall":0.0080,
        "category diversity":0}

### k=50, lambda_text=0.1, l2_reg=1

In [13]:
# Instantiate HFT model
hft_l21_lamText01_k50 = cornac.models.HFT(
    k=50,
    max_iter=10,
    grad_iter=5,
    l2_reg=1,
    lambda_text=0.1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=clothing_ratio_split, models=[hft_l21_lamText01_k50], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/2200 [00:00<?, ?it/s]

Ranking:   0%|          | 0/1868 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.7459 | 1.1147 | 1.0558 | 0.5119 | 0.0003 | 0.0021 | 0.0024 |  0.0009 |  0.0019 |       0.0001 |    0.0064 |   39.2217 |   2.8857



In [14]:
data10 = {"model":"HFT",
        "data":"amazon clothing",
        "k":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.1,
        "l2_reg":1,
        "MAE":0.7459,"MSE":1.1147,"RMSE":1.0558,"AUC":0.5119,"F1":0.0003,"MAP":0.0021,"MRR":0.0024,"NCRR":0.0009,"NDCG":0.0019,"Precision":0.0001,"Recall":0.0064,
        "category diversity":0}

In [21]:
data = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10]
df = pd.DataFrame(columns = data1.keys())
i=0
for d in data:
    df = pd.concat([df, pd.DataFrame(d,columns = list(d.keys()),index=[i])], ignore_index=False)
    i += 1

In [22]:
df

,model,data,k,max_iter,grad_iter,lambda_text,l2_reg,MAE,MSE,RMSE,AUC,F1,MAP,MRR,NCRR,NDCG,Precision,Recall,category diversity
0,HFT,amazon clothing,10,10,5,0.01,0.001,0.7492,1.2143,1.1020,0.5053,0.0003,0.0020,0.0024,0.0009,0.0021,0.0002,0.0075,0
1,HFT,amazon clothing,50,10,5,0.01,0.001,0.7426,1.1828,1.0876,0.5061,0.0004,0.0019,0.0024,0.0009,0.0022,0.0002,0.0083,0
2,HFT,amazon clothing,100,10,5,0.01,0.001,0.7508,1.2095,1.0998,0.5035,0.0003,0.0020,0.0024,0.0009,0.0019,0.0001,0.0062,0
3,HFT,amazon clothing,200,10,5,0.01,0.001,0.7359,1.1497,1.0723,0.5071,0.0003,0.0019,0.0024,0.0008,0.0020,0.0002,0.0072,0
4,HFT,amazon clothing,300,10,5,0.01,0.001,0.7391,1.1584,1.0763,0.5036,0.0003,0.0019,0.0024,0.0009,0.0020,0.0002,0.0070,0
5,HFT,amazon clothing,50,10,5,0.10,0.001,0.7482,1.1211,1.0588,0.5110,0.0003,0.0021,0.0025,0.0009,0.0018,0.0001,0.0059,0
6,HFT,amazon clothing,50,10,5,1.00,0.001,0.8140,1.3782,1.1739,0.5359,0.0022,0.0069,0.0078,0.0054,0.0135,0.0011,0.0469,0
7,HFT,amazon clothing,50,10,5,0.10,0.010,0.7520,1.1127,1.0549,0.5113,0.0003,0.0021,0.0024,0.0009,0.0018,0.0001,0.0064,0
8,HFT,amazon clothing,50,10,5,0.10,0.100,0.7397,1.1815,1.0870,0.5096,0.0004,0.0020,0.0024,0.0009,0.0021,0.0002,0.0080,0
9,HFT,amazon clothing,50,10,5,0.10,1.000,0.7459,1.1147,1.0558,0.5119,0.0003,0.0021,0.0024,0.0009,0.0019,0.0001,0.0064,0


In [23]:
df.to_csv('/Users/pigr/Desktop/cornac/cornac/dataset/hft_amazon_clothing.csv',header=-1,index=0)